# <center>ディープラーニング入門</center>
# <center>A LSTM in Wonderland</center>


In [22]:

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [23]:
import numpy as np

# 1. 文字単位での文章生成

今回はLSTMで文章生成してみます。Kerasのexampleを少し改造したモデルを用意しました。

## 1-1. テキストデータのクリーニング

青空文庫から不思議の国のアリスのテキストファイルをダウンロードしましょう。

https://www.aozora.gr.jp/cards/001393/card57320.html

解凍したファイルのうち、alicewa_fushigino_kunide.txtをGoogle Driveのaliceフォルダに入れておきます。

In [24]:
file_path = './text/alicewa_fushigino_kunide.txt'
    
with open(file_path, 'r', encoding='shift-jis') as text_file:
    strings = text_file.read()
    
paras = strings.splitlines()

開いたテキストを見てみます：

In [25]:
paras

['アリスはふしぎの国で',
 'ALICE IN WONDERLAND',
 'ルイス・キャロル\u3000Lewis Carroll',
 '大久保ゆう訳',
 '',
 '-------------------------------------------------------',
 '【テキスト中に現れる記号について】',
 '',
 '《》：ルビ',
 '（例）階段《かいだん》',
 '',
 '［＃］：入力者注\u3000主に外字の説明や、傍点の位置の指定',
 '（例）［＃挿絵１（fig57320_01.png、横511×縦762）入る］',
 '-------------------------------------------------------',
 '',
 '［＃４字下げ］アリスはふしぎの国で［＃「アリスはふしぎの国で」は大見出し］',
 '',
 '［＃ここから３字下げ］',
 'ぜんぶ\u3000きんきら\u3000ごごのこと',
 '\u3000ゆるーり\u3000すいすい\u3000ぼくらは\u3000すすむ',
 '２ほんの\u3000オールで\u3000ぎこちなく',
 '\u3000ほそい\u3000かいなで\u3000こいでゆく',
 'しろい\u3000おててが\u3000かっこうだけは',
 '\u3000うねうね\u3000つづく\u3000さきを\u3000しめす',
 '',
 'おお\u3000きびしい\u3000３にんの\u3000ひめ！',
 '\u3000よりによって\u3000こんなとき\u3000すてきなてんきに',
 'いきも\u3000きれぎれ\u3000はね１ぽん\u3000びくとも',
 '\u3000させられないのに\u3000おはなしを\u3000せがむなんて！',
 'でも\u3000しゃべるくちは\u3000ひとつしかないんだよ',
 '\u3000３にん\u3000いっしょに\u3000いわれても……！',
 '',
 'ふんぞりかえる\u3000１のひめ\u3000こっちを',
 '\u3000にらんで\u3000さしず\u3000「おはじめなさい」',
 'おしとやかにも\u3000２のひめの\u3000おのぞみは',
 '\u3000「すっからかんな\u3000おはなしが

色々と本文とは関係ないものが混ざっていますので、それらを取り除きます。こういうことは手作業でしません（自動化しにくく最後まで残ってしまうちょっとしたゴミだけ、手作業で除去しましょう）：

In [26]:
import re
import codecs

save_path = './output/alice.txt'
    
end = 0
# あとがき以下削除
for i in range(150):
    if paras[-i]=='子どものみなみなさま':
        end=i+3
paras = paras[:-end]

# 本文以前の冒頭を削除     
for i, p in enumerate(paras):
    if p[:4]=='----':
        end = i+3
paras = paras[end:]

# 見出し等削除して保存
r = codecs.open(save_path, 'w', 'utf8')
marks = re.compile(r'［[^［]*］|《[^《]*》|｜|\u3000')
for p in paras:
    q = re.sub(marks, '', p)
    r.write(q)  
r.close()

特殊な挿入物は正規表現で削除していきます。

ではクリーニング後のテキストを確認してみましょう：

In [27]:
import io

with io.open(save_path, encoding='utf-8') as f:
    text = f.read().lower()

print('コーパス長:', len(text))

コーパス長: 65766


In [28]:
text

'ぜんぶきんきらごごのことゆるーりすいすいぼくらはすすむ２ほんのオールでぎこちなくほそいかいなでこいでゆくしろいおててがかっこうだけはうねうねつづくさきをしめすおおきびしい３にんのひめ！よりによってこんなときすてきなてんきにいきもきれぎれはね１ぽんびくともさせられないのにおはなしをせがむなんて！でもしゃべるくちはひとつしかないんだよ３にんいっしょにいわれても……！ふんぞりかえる１のひめこっちをにらんでさしず「おはじめなさい」おしとやかにも２のひめのおのぞみは「すっからかんなおはなしがあるといいな！」それでいて３のひめはかたるそばから１ぷんに１どはちゃちゃいれるしやがてたちまちしずまりかえりおもいえがいてたどっていくのはびっくりどきどきふしぎのせかいをゆめの子どもがどんどんゆくさまとりやけものとおしゃべりしながら――じぶんでもなんだかゆめうつつするうちものがたりいきづまりおもいつきもそこついてそこでへとへとふらふらのためなんとかひとまずうちきろうと「つづきはまたこんど――」「いまがこんど！」とおおごえではしゃがれるかくしてふしぎのくにのおはなしがうまれこうしてゆっくりひとつずつへんてこなできごとがひねりだされて――そしてここまではなしはおしまいふねをおうちへむけるにぎやかないちどううしろでおひさましずんでいくよアリス！おとぎばなしをどうぞそれからやさしいおててでそなえてほしいおもいでというひみつのいとでぬいこまれたこどものときのゆめにいまはもうしおれてしまったはるかとおくでつんだはなわに１ひゅうんとウサギ穴へアリスはあっきあきしてきた、木かげで、お姉さまのそばですわってるのも、何もしないでいるのも――ちらちらお姉さまの読んでる本をのぞいてみても、さし絵もかけ合いもない、「なら本のねうちって何、」とアリスは思う、「さし絵もかけ合いもないなんて。」だから物思いにふけるばっかり（といってもそれなり、だって日ざしぽかぽかだとぼんやりねむくなってくるし）、デイジーの花輪作りはわざわざ立ち上がって花をつむほど楽しいものなのかしら――そこへふといきなり赤い目の白ウサギが１羽そばをかけぬける。たいして目を引くようなところもないから、アリスにしてもさほどとんでもないとも感じないまま、聞こえてくるウサギのひとりごと。「およよ！およよ！ちこくでおじゃる！」（あとになって思い返すと、ここでふ

無事綺麗になりました。

## 1-2. 訓練データの準備

テキストに現れる固有の文字を、setにすることで数え上げてみましょう：

In [29]:
chars = sorted(list(set(text)))

print('全文字数:', len(chars))

全文字数: 712


In [30]:
chars

['!',
 '0',
 '1',
 '2',
 '?',
 '×',
 '―',
 '…',
 '※',
 '→',
 '、',
 '。',
 '々',
 '〆',
 '〈',
 '〉',
 '「',
 '」',
 '『',
 '』',
 '〜',
 'あ',
 'ぃ',
 'い',
 'ぅ',
 'う',
 'ぇ',
 'え',
 'お',
 'か',
 'が',
 'き',
 'ぎ',
 'く',
 'ぐ',
 'け',
 'げ',
 'こ',
 'ご',
 'さ',
 'ざ',
 'し',
 'じ',
 'す',
 'ず',
 'せ',
 'ぜ',
 'そ',
 'ぞ',
 'た',
 'だ',
 'ち',
 'ぢ',
 'っ',
 'つ',
 'づ',
 'て',
 'で',
 'と',
 'ど',
 'な',
 'に',
 'ぬ',
 'ね',
 'の',
 'は',
 'ば',
 'ぱ',
 'ひ',
 'び',
 'ぴ',
 'ふ',
 'ぶ',
 'ぷ',
 'へ',
 'べ',
 'ぺ',
 'ほ',
 'ぼ',
 'ぽ',
 'ま',
 'み',
 'む',
 'め',
 'も',
 'ゃ',
 'や',
 'ゅ',
 'ゆ',
 'ょ',
 'よ',
 'ら',
 'り',
 'る',
 'れ',
 'ろ',
 'わ',
 'を',
 'ん',
 'ァ',
 'ア',
 'ィ',
 'イ',
 'ウ',
 'ェ',
 'エ',
 'ォ',
 'オ',
 'カ',
 'ガ',
 'キ',
 'ギ',
 'ク',
 'グ',
 'ケ',
 'ゲ',
 'コ',
 'ゴ',
 'サ',
 'ザ',
 'シ',
 'ジ',
 'ス',
 'ズ',
 'セ',
 'ゼ',
 'ソ',
 'タ',
 'ダ',
 'チ',
 'ッ',
 'ツ',
 'テ',
 'デ',
 'ト',
 'ド',
 'ナ',
 'ニ',
 'ネ',
 'ノ',
 'ハ',
 'バ',
 'パ',
 'ヒ',
 'ビ',
 'ピ',
 'フ',
 'ブ',
 'プ',
 'ヘ',
 'ベ',
 'ペ',
 'ホ',
 'ボ',
 'ポ',
 'マ',
 'ミ',
 'ム',
 'メ',
 'モ',
 'ャ',
 'ヤ',
 'ュ',
 'ョ',
 'ヨ',
 'ラ',
 'リ'

これらの文字に、対応する数字を割り当てておく必要があります。両者の対応を定める辞書を作っておきます：

In [31]:
char2indices = dict((c, i) for i, c in enumerate(chars))
indices2char = dict((i, c) for i, c in enumerate(chars))

では文字をこの数字に置き換えながら訓練データを作っておきましょう。まず入力データは、テキスト全体を`maxlen`の長さに切り出したsentenceの集まりです。切り出しは`step`数をストライドにして行います。各sentenceに対して次の文字(next character)を出力させるようにLSTMを学習させます。

In [32]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('文の数:', len(sentences))

文の数: 21909


出来上がった入力文は、中身を見れば何を作ったのかが一目瞭然です：

In [33]:
sentences

['ぜんぶきんきらごごのことゆるーりすいすいぼくらはすすむ２ほんのオールでぎこちなく',
 'きんきらごごのことゆるーりすいすいぼくらはすすむ２ほんのオールでぎこちなくほそい',
 'らごごのことゆるーりすいすいぼくらはすすむ２ほんのオールでぎこちなくほそいかいな',
 'のことゆるーりすいすいぼくらはすすむ２ほんのオールでぎこちなくほそいかいなでこい',
 'ゆるーりすいすいぼくらはすすむ２ほんのオールでぎこちなくほそいかいなでこいでゆく',
 'りすいすいぼくらはすすむ２ほんのオールでぎこちなくほそいかいなでこいでゆくしろい',
 'すいぼくらはすすむ２ほんのオールでぎこちなくほそいかいなでこいでゆくしろいおてて',
 'くらはすすむ２ほんのオールでぎこちなくほそいかいなでこいでゆくしろいおててがかっ',
 'すすむ２ほんのオールでぎこちなくほそいかいなでこいでゆくしろいおててがかっこうだ',
 '２ほんのオールでぎこちなくほそいかいなでこいでゆくしろいおててがかっこうだけはう',
 'のオールでぎこちなくほそいかいなでこいでゆくしろいおててがかっこうだけはうねうね',
 'ルでぎこちなくほそいかいなでこいでゆくしろいおててがかっこうだけはうねうねつづく',
 'こちなくほそいかいなでこいでゆくしろいおててがかっこうだけはうねうねつづくさきを',
 'くほそいかいなでこいでゆくしろいおててがかっこうだけはうねうねつづくさきをしめす',
 'いかいなでこいでゆくしろいおててがかっこうだけはうねうねつづくさきをしめすおおき',
 'なでこいでゆくしろいおててがかっこうだけはうねうねつづくさきをしめすおおきびしい',
 'いでゆくしろいおててがかっこうだけはうねうねつづくさきをしめすおおきびしい３にん',
 'くしろいおててがかっこうだけはうねうねつづくさきをしめすおおきびしい３にんのひめ',
 'いおててがかっこうだけはうねうねつづくさきをしめすおおきびしい３にんのひめ！より',
 'てがかっこうだけはうねうねつづくさきをしめすおおきびしい３にんのひめ！よりによっ',
 'っこうだけはうねうねつづくさきをしめすおおきびしい３にんのひめ！よりによってこん',
 'だけはうねうねつづくさきをしめすおおきびしい３にんのひめ！よりによってこんなとき',
 'うねうねつづくさ

訓練に使うために、このsentenceたちを数字の並びに置き換えておきます。

In [34]:
x = np.zeros((len(sentences), maxlen))

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t] = char2indices[char]

In [35]:
x.shape

(21909, 40)

出力はsoftmax層で予測するので、対応する文字のインデックスではなくone-hotッベクトルにしておきます。

In [36]:
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    y[i, char2indices[next_chars[i]]] = 1

In [37]:
y.shape

(21909, 712)

## 1-3. LSTMの学習

モデルはシンプルなものを作っておきます。深くしたりLSTM以外を使うことなどは各自で試してみてください。

In [38]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM, Embedding
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(input_dim=len(chars), output_dim=128, input_length=maxlen))
model.add(LSTM(128))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.summary()

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 128)           91136     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 712)               91848     
_________________________________________________________________
activation (Activation)      (None, 712)               0         
Total params: 314,568
Trainable params: 314,568
Non-trainable params: 0
_________________________________________________________________


訓練中に生成文をサンプリングするためのcall backを用意しておきます：

In [39]:
import random
import sys

# 様々な温度の予測確率分布からランダムに予測文字をサンプリングすることで、生成文の多様性を変える
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# 今回は生成のseedは固定
start_index = random.randint(0, len(text) - maxlen - 1)
sentence = text[start_index: start_index + maxlen]
print('次のseed文に続く文を生成する: ')
print(sentence)

def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen))
            for t, char in enumerate(sentence):
                x_pred[0, t] = char2indices[char]

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices2char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

次のseed文に続く文を生成する: 
リスがあたりをながめまわしても、草花あれど、都合よく飲み食いできそうなものはその


では訓練しましょう：

In [ ]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
169/172 [============================>.] - ETA: 0s - loss: 4.2621
----- Generating text after Epoch: 0
----- diversity: 0.2
リスがあたりをながめまわしても、草花あれど、都合よく飲み食いできそうなものはそのはあっていたんだけどうしたいっていたいたんだっていたんで、そうしたりしたんと、そこできたんだっていたんだったいうしたんだから、そこにあっていたんで、そうなんでもうちゃいたんだけどうしたんで、そうなんであっていたんで。」「そうしたうしたいたんだっていたんで、そうなんと、そうしたんだけどうしたんで、そこでもうしたんで、そうしたんできっていたんで、そうなんで、そうしたいっとしたんでもうしたんで、そうなんでもうちゃいっていたんで、そこできたんだから、そうなんでもうしたんで、こうなんでもうちっていたいから。」とアリス。「もうしたんだっていたんで、そうしたんだんで、そこでもうしたんだんだけどうしたんで、そうなんでもうちゃいたんで。」とアリス。「そうしたいのことに、それないっていたんだけどうしたんで、そうなんでもうしたんで、そうしたこと、そうなんで、そこでもうちゃいたんだから、そうなんでもうちゃいたんで
----- diversity: 0.5
リスがあたりをながめまわしても、草花あれど、都合よく飲み食いできそうなものはそのはきゃるうしどうしちゃいっていったからしていたいたからわったんで。」とそうしたっところにもうとおっていたい、ころ、それでもうなんで、」ところにおわいていっていう。」とでも、それでもあっていっているうしていたたくらときたんでウイン。「そうとまっていううと、そうのあたちゃいっともうもうんで、そんでこっちになんだったくめいくまるっていたいどうしたりなんでってきたりもおまっていたずどうっとしたんと、こんないわちょっていったいのに、あっていたうしたんかり、」とおじちゃんで、ちょったんなこと、そそこの考えんで、それでなんだけどうちゃ、」と思いのもここにそうにある。」と言うしたっていたこうどうしうしていっていたいっていたんで、「もうと。」「っていって、そうしたいったくったいたうと、これなかごとつけえくいうわっていくしたいたんでもう

171/172 [============================>.] - ETA: 0s - loss: 2.8021
----- Generating text after Epoch: 4
----- diversity: 0.2
リスがあたりをながめまわしても、草花あれど、都合よく飲み食いできそうなものはそのあと、おめにすると、こう言うとなってる。」「そうしてこのはあたりにはきってるだけで、おってことにうしてると、このあと思ってるから、」とアリスは、だっててこのかたんにゃんだけど、」とアリスは思いつけて、「それはあって、「それにつからいているのはいいのは、」とアリスは思いつから、おさばきの場のはないと、それにうちにないのは見えるうとしてね、「そういうのはすると、こうになるのは何からいてね。「それにつからないのはすっとして、」とアリスは思ってると、「それにううしてことはなくてよ！」とアリス。「それはおいえ。」とアリスは思いつから、」とアリスは思いつらあってる。」とアリスは思いつら、あたくしののがたんだから。」とアリスは思いつら、あたくしのにはなくて、そのあと見つめるようにすると、そうしてそれもうちまえ、」とアリスは思いつけるのはあってるとしてね、「それもおれはおいつも、だからおしにもうちへえんと
----- diversity: 0.5
リスがあたりをながめまわしても、草花あれど、都合よく飲み食いできそうなものはそのかれをとうんでするのはすか。」とアリスは言うアリス、「だれも言いえる。」とアリスはおずしはってコがあれておじゃるって。」（アリスはちっとウミガメフーミの声にはずあると、」とアリスはおずさまえのないのはすると、またものがあの子たぶり。「だれもおしゃべかって、「こんなことをやるのは」とアリスは思うアリス。「だれでするとしてね、「それもうちのないのはヨっぱりはきって？」とアリスは思って答えると、このは何もんだからね。」とアリスは思いつから、あたくしてそれていてかけわればからね！」「そう言いささたして。」とアリスは言うして、「それだらお手にはびたりと思いつから、」とアリスは思いつらいってね、「それももうなくているのはすっとしてね。」としいやつきらせるのはなったけど、そうしらなくてきがらようにそれももどってくるようかな！」とアリスはおれいにはずさん